# Dos dados

Existem duas fontes de dados para a classificação do setor destino dos avisos que chegam a PGM:

1. Os dados antigos que foram tratados por Nailton, compreendidos entre abril e dezembro de 2024 (15952 avisos)

    * *Está presente apenas no arquivo pgm-dataset.csv*

2. Os dados novos tratados por Esdras, compreendidos entre janeiro e maio de 2025 (cerca de 34000 avisos)

    * *O setor destino está na planilha tramitacoes_pje_janeiro_maio_2025.xls e os textos estão em df_avisos.csv*

<h3> <b>Objetivo:</b> Criar 3 DataFrames diferentes com os avisos antigos, novos e uma junção dos dois. </h3>

* Eles devem ter os mesmos atributos normalizados (mesmo formato) em todos os DataFrames.
* São os atributos: id_aviso, data_hora, teor_texto, setor_destino.
* *OBS.*: o atributo data_hora não tem tanta significância, uma vez que nos dados antigos ela virá da coluna 'dataAviso' e nos dados novos ela virá da coluna 'dataenvioDestino' presente no excel.

# 1. Criando os dataframes

In [1]:
import pandas as pd

df_antigo = pd.read_csv('data/raw/pgm-dataset.csv')
df_novo_1 = pd.read_csv('data/raw/df_avisos.csv')
df_novo_2 = pd.read_excel('data/raw/tramitacoes_pje_janeiro_maio_2025.xls')

## 1.1 - Primeiro DataFrame (2024)

In [2]:
# Apenas selecionando os dados dos avisos antigos para formar o primeiro dataframe

df_avisos_2024 = pd.DataFrame(data=df_antigo[['idAviso', 'dataAviso', 'teorTexto', 'setorDestino']].values,
                              columns=['id_aviso', 'data_hora', 'teor_texto', 'setor_destino'])

df_avisos_2024.head(3)

,id_aviso,data_hora,teor_texto,setor_destino
0,18612632.0,2024-06-21 05:00:40,PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,Procuradoria da Sa�de
1,18629258.0,2024-06-25 23:59:59,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Procuradoria Administrativa
2,18629422.0,2024-06-18 09:27:16,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,CARTORIO


## 1.2 - Segundo DataFrame (2025)

É necessário:
1. Filtrar do df_novo_2 apenas aqueles que tem um setor destino válido e recuperar os id's válidos
2. Filtrar do df_novo_1 apenas aqueles que tem id válido (obtido no passo 1)
3. Fazer o Merge dos dos Dataframes e filtrar apenas as colunas necessárias

In [3]:
# 1. Filtrar do df_novo_2 apenas aqueles que tem um setor destino válido e recuperar os id's válidos
print(f'Temos {df_novo_2['setordestino'].isna().sum()} documentos com destino inválido!')
df_labels_validos = df_novo_2[df_novo_2['setordestino'].notnull()]
ids_validos = df_labels_validos['idavisopje'].tolist()

# 2. Filtrar do df_novo_1 apenas aqueles que tem id válido (obtido no passo 1)
df_textos_filtrados = df_novo_1[df_novo_1['id_aviso'].isin(ids_validos)]
df_textos_filtrados.rename(columns={'id_aviso':'idavisopje'}, inplace=True) # Renomeando para fazer o merge

# 3. Fazer o Merge dos dos Dataframes e filtrar apenas as colunas necessárias
df_merge = pd.merge(df_labels_validos, df_textos_filtrados, on='idavisopje')
df_avisos_2025 = df_merge[['idavisopje', 'dataenvioDestino', 'teor_texto', 'setordestino']].copy()
df_avisos_2025.rename(columns={df_avisos_2025.columns[i]: df_avisos_2024.columns[i] for i in range(4)}, inplace=True)
df_avisos_2025['data_hora'] = df_avisos_2025['data_hora'].apply(lambda x: x.split('.')[0])

Temos 0 documentos com destino inválido!


In [4]:
df_avisos_2025.head(3)

,id_aviso,data_hora,teor_texto,setor_destino
0,21303687,2025-01-02 09:34:16,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Procuradoria Administrativa
1,21303704,2025-01-02 09:31:16,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Procuradoria Administrativa
2,21305478,2025-01-02 09:32:58,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Procuradoria Administrativa


# 2. Tratando os DataFrames

## 2.1 - Removendo valores nulos de setor_destino e teor_texto

In [5]:
# Df Antigo
len_df_antes = len(df_avisos_2024)
df_avisos_2024.dropna(subset=['teor_texto', 'setor_destino'], inplace=True, ignore_index=True)
len_df_depois = len(df_avisos_2024)
print(f'Antes o DataFrame tinha: {len_df_antes} objetos, agora tem {len_df_depois}. Uma redução de {(len_df_antes-len_df_depois)/len_df_antes*100:.2f}%')

df_avisos_2024.isnull().sum()

Antes o DataFrame tinha: 15942 objetos, agora tem 15356. Uma redução de 3.68%


id_aviso         0
data_hora        0
teor_texto       0
setor_destino    0
dtype: int64

In [6]:
# Df Novo
len_df_antes = len(df_avisos_2025)
df_avisos_2025.dropna(subset=['teor_texto', 'setor_destino'], inplace=True, ignore_index=True)
len_df_depois = len(df_avisos_2025)
print(f'Antes o DataFrame tinha: {len_df_antes} objetos, agora tem {len_df_depois}. Uma redução de {(len_df_antes-len_df_depois)/len_df_antes*100:.2f}%')

df_avisos_2025.isnull().sum()

Antes o DataFrame tinha: 34903 objetos, agora tem 34903. Uma redução de 0.00%


id_aviso         0
data_hora        0
teor_texto       0
setor_destino    0
dtype: int64

## 2.2 - Tratando os setores destino

In [12]:
# Camel Case
def to_camel_case(text):
    text = text.lower()
    text = text.split()
    return ' '.join([word.capitalize() if len(word) > 2 else word for word in text])

df_avisos_2024['setor_destino'] = df_avisos_2024['setor_destino'].apply(to_camel_case)
df_avisos_2025['setor_destino'] = df_avisos_2025['setor_destino'].apply(to_camel_case)

In [20]:
# Corrigindo erros de codificação

dict_swap = {'Departamento de C�lculos e Contabilidade': 'Departamento de Cálculos e Contabilidade',
             'Procuradoria da Sa�de': 'Procuradoria da Saúde',
             'Setor de Administra�?o Geral': 'Setor de Administração Geral',
             'Procuradoria de Recupera�?o da D�vida Ativa': 'Procuradoria de Recuperação da Dívida Ativa'}

df_avisos_2024['setor_destino'] = df_avisos_2024['setor_destino'].replace(dict_swap)

# 3. Salvando os DataFrames

## 3.1 - Concatenando os dois dataframes em um só

In [36]:
df_avisos_concat = pd.concat([df_avisos_2024, df_avisos_2025], ignore_index=True)
df_avisos_concat.head(3)

,id_aviso,data_hora,teor_texto,setor_destino
0,18612632.0,2024-06-21 05:00:40,PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,Procuradoria da Saúde
1,18629258.0,2024-06-25 23:59:59,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Procuradoria Administrativa
2,18629422.0,2024-06-18 09:27:16,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Cartorio


## 3.2 - Salvando os DataFrames

In [37]:
import os

def save_csv(df, path):
    if os.path.exists(path):
        print(f'Arquivo já existe: {path}')
    else:
        df.to_csv(path, index=False)
        print(f'DataFframe salvo em: {path}')

saving_path = 'data/processed'

save_csv(df_avisos_2024, os.path.join(saving_path, 'df_avisos_antigos.csv'))
save_csv(df_avisos_2025, os.path.join(saving_path, 'df_avisos_novos.csv'))
save_csv(df_avisos_concat, os.path.join(saving_path, 'df_avisos_completos.csv'))

DataFframe salvo em: data/processed/df_avisos_antigos.csv
DataFframe salvo em: data/processed/df_avisos_novos.csv
DataFframe salvo em: data/processed/df_avisos_completos.csv
